# Adjust index returns for inflation to get real returns

In [ ]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
wb <- WorldBank()
indices <- Indices()

### get rate of inflation from the WorldBank data-set

In [ ]:
# see example: https://nbviewer.jupyter.org/github/shyams80/plutons/blob/master/docs-R/WorldBank.ipynb

wb$Meta() %>%
    filter(COUNTRY_NAME == "India" & INDICATOR_NAME %like% "CPI%") %>%
    select(INDICATOR_NAME, COUNTRY_KEY, INDICATOR_KEY) %>%
    print(n=Inf)

In [ ]:
# we need the CPI Price, % y-o-y, nominal, seas. adj.
cpiInflation <- wb$TimeSeries() %>%
    filter(COUNTRY_KEY == 135 & INDICATOR_KEY == 6) %>%
    arrange(YEAR) %>%
    select(YEAR, VALUE) %>%
    as.data.frame

print(cpiInflation)

### get NIFTY 50 total-return index returns from the Indices data-set

In [ ]:
indexName <- 'NIFTY 50 TR'
nifty50Dt <- indices$NseTimeSeries() %>%
    filter(NAME == indexName) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

annRets <- 100*annualReturn(xts(nifty50Dt$CLOSE, nifty50Dt$TIME_STAMP))

df1 <- data.frame(annRets)
df1$YEAR <- year(index(annRets))

print(df1)

In [ ]:
# merge the two

allData <- merge(df1, cpiInflation, by='YEAR')
names(allData) <- c('YEAR', 'INDEX', 'INFLATION')
allData$REAL <- allData$INDEX - allData$INFLATION
print(allData)

In [ ]:
# plot the data

mDf <- melt(allData, id='YEAR')
ggplot(mDf, aes(x=YEAR, y=value, fill=variable)) +
    theme_economist() +
    geom_bar(stat="identity", position=position_dodge()) +
    scale_x_continuous(labels=allData$YEAR, breaks=allData$YEAR) +
    geom_text_repel(aes(label= round(value, 2)), position = position_dodge(0.9)) +
    labs(x='', y='(%)', fill='', title=sprintf("%s Gross vs. Real (inflation-adjusted) Returns", indexName), 
         subtitle="Annual Returns") +
    annotate("text", x=max(allData$YEAR), y=min(mDf$value), 
             label = "@StockViz", hjust=1.1, vjust=-1.1, 
             col="white", cex=6, fontface = "bold", alpha = 0.8)  

In [ ]:
### calculate rolling 5-year inflation adjusted retuns

rollingRets <- 100*rollapply(allData$REAL/100, 5, Return.cumulative)
print(rollingRets)
summary(rollingRets)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)